### Imoprting required libraries

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import os
import cv2

Using TensorFlow backend.


Setting Path 

In [2]:
train_path='../input/train/train'
test_path='../input/test/test'

Reading Train files

In [3]:
train = pd.read_csv("../input/train.csv")
train.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


#### Storing Images and Id in Different arrays (Training Data)

In [4]:
id=train['id'].values
cactus=train['has_cactus'].values

train=[]
X=[]
Y=[]
a=0

for i in tqdm(sorted(os.listdir(train_path))):
    path=os.path.join(train_path,i)
    i=cv2.imread(path,cv2.IMREAD_COLOR)
    X.append(i)
    train.append([np.array(cactus),cactus[a]])
    a=a+1

train=np.array(train)
Y=train[:,1]
train=train[:,0]
X=np.array(X)

X.shape

X=X/255
train=train/255

100%|██████████| 17500/17500 [00:29<00:00, 588.12it/s]


#### Storing Images in Array (Testing Data)

In [5]:
test1=[]
X_test=[]

for i in tqdm(os.listdir(test_path)):
    id=i
    path=os.path.join(test_path,i)
    i=cv2.imread(path,cv2.IMREAD_COLOR)
    X_test.append(i)
    test1.append([np.array(i),id])

X_test=np.array(X_test)
X_test.shape
test1=np.array(test1)
id_test=test1[:,1]
test1=test1[:,0]
test1.shape

X_test=X_test/255
test1=test1/255

100%|██████████| 4000/4000 [00:05<00:00, 671.33it/s]


### Creating Model

In [6]:
model = Sequential()
model.add(Conv2D(filters=128,kernel_size=2,padding="same",activation="relu",input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=2,strides=1))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2,strides=1))
model.add(Dropout(0.2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2,strides=1))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.7))
model.add(Dense(1,activation="sigmoid"))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)  

### Training Model

In [7]:
from keras.callbacks import EarlyStopping
el=EarlyStopping(min_delta=0.001,patience=5)
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
h=model.fit(X,Y,batch_size=256,validation_split=0.2,epochs=100,callbacks=[el])

Instructions for updating:
Use tf.cast instead.
Train on 14000 samples, validate on 3500 samples
Epoch 1/100
14000/14000 [==============================] - 6s 430us/step - loss: 0.4878 - acc: 0.7525 - val_loss: 0.2828 - val_acc: 0.8757
Epoch 2/100
14000/14000 [==============================] - 3s 195us/step - loss: 0.2986 - acc: 0.8014 - val_loss: 0.2372 - val_acc: 0.9371
Epoch 3/100
14000/14000 [==============================] - 3s 194us/step - loss: 0.2819 - acc: 0.8767 - val_loss: 0.2014 - val_acc: 0.9451
Epoch 4/100
14000/14000 [==============================] - 3s 193us/step - loss: 0.2650 - acc: 0.8866 - val_loss: 0.1925 - val_acc: 0.9440
Epoch 5/100
14000/14000 [==============================] - 3s 210us/step - loss: 0.2554 - acc: 0.8881 - val_loss: 0.1686 - val_acc: 0.9546
Epoch 6/100
14000/14000 [==============================] - 3s 205us/step - loss: 0.2432 - acc: 0.8948 - val_loss: 0.1546 - val_acc: 0.9603
Epoch 7/100
14000/14000 [==============================] - 3s 195us/s

### Predicting

In [8]:
pred=model.predict(X_test)
ids=[]
label=[]
a=0
for i in tqdm(os.listdir(test_path)):
    id=i
    ids.append(id)
    label.append(pred[a])
    a=a+1

label=np.array(label,dtype='float64')
out=pd.DataFrame({'id': ids,'has_cactus':label[:,0]})

out.to_csv('submission.csv',index=False,header=True)

100%|██████████| 4000/4000 [00:00<00:00, 791938.45it/s]


### Help Me Improve The Accuracy I Am New :)